In [1]:
import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('circuits'), '..')))

import argparse
import re
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import circuits.toff_7T_decomposition as toff
import circuits.dj_circuit as dj
import qcm_sim as sim

# suppress qiskit 1.0 deprecation warnings:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

In [2]:
# number of qubits;
n = 4

In [3]:
print(f"Quantum Circuit:\n")

# Test deutsch jozsa on constant:
qc = dj.run_deutsch_jozsa_test(n_qubits=n, oracle_type='constant', constant_output=0)

# Simulate the circuit after decomposition
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=4096)
result = job.result()
counts = result.get_counts()

print("\nQiskit Simulation Results:")
#print(counts)
percentages = [(k, v / sum(counts.values()))
                    for k, v in counts.items()]
print(f"Percentages: {percentages}\n")

Quantum Circuit:

Using a CONSTANT oracle that always returns 0
Quantum Circuit:

     ┌───┐┌───┐     ┌─┐                   
q_0: ┤ H ├┤ H ├─────┤M├───────────────────
     ├───┤├───┤     └╥┘┌─┐                
q_1: ┤ H ├┤ H ├──────╫─┤M├────────────────
     ├───┤├───┤      ║ └╥┘┌─┐             
q_2: ┤ H ├┤ H ├──────╫──╫─┤M├─────────────
     ├───┤├───┤      ║  ║ └╥┘┌─┐          
q_3: ┤ H ├┤ H ├──────╫──╫──╫─┤M├──────────
     ├───┤├───┤┌───┐ ║  ║  ║ └╥┘┌───┐┌───┐
q_4: ┤ H ├┤ S ├┤ S ├─╫──╫──╫──╫─┤ H ├┤ H ├
     └───┘└───┘└───┘ ║  ║  ║  ║ └───┘└───┘
c: 4/════════════════╩══╩══╩══╩═══════════
                     0  1  2  3            

Conclusion: f(x) is CONSTANT.

Qiskit Simulation Results:
Percentages: [('0000', 1.0)]



In [4]:
# Apply the 7T decomposition if applicable
qc_with_decomposition = toff.apply_toff_via_7t_decomposition(qc)

print("With decomposition:")
print(qc_with_decomposition)

qasm_code = qc_with_decomposition.qasm()

# parent directory:
pardir = os.path.abspath(os.path.join(os.path.dirname('circuits'), '..'))
qasm_dir = os.path.join(pardir,'qasm_files')

filename = f'dj_{n}_constant.qasm'
qasm_path = os.path.join(qasm_dir,filename)

# Replace 'tdg' with 's; s; s; t;'
modified_qasm_code = re.sub(r'\btdg\b', 's;\ns;\ns;\nt;', qasm_code)

with open(qasm_path, "w") as f:
    f.write(modified_qasm_code)


With decomposition:
      ┌───┐┌───┐     ┌─┐                                     
 q_0: ┤ H ├┤ H ├─────┤M├─────────────────────────────────────
      ├───┤├───┤     └╥┘┌─┐                                  
 q_1: ┤ H ├┤ H ├──────╫─┤M├──────────────────────────────────
      ├───┤├───┤      ║ └╥┘┌─┐                               
 q_2: ┤ H ├┤ H ├──────╫──╫─┤M├───────────────────────────────
      ├───┤├───┤      ║  ║ └╥┘┌─┐                            
 q_3: ┤ H ├┤ H ├──────╫──╫──╫─┤M├────────────────────────────
      ├───┤├───┤      ║  ║  ║ └╥┘┌─┐                         
 q_4: ┤ H ├┤ H ├──────╫──╫──╫──╫─┤M├─────────────────────────
      ├───┤├───┤      ║  ║  ║  ║ └╥┘┌─┐                      
 q_5: ┤ H ├┤ H ├──────╫──╫──╫──╫──╫─┤M├──────────────────────
      ├───┤├───┤      ║  ║  ║  ║  ║ └╥┘┌─┐                   
 q_6: ┤ H ├┤ H ├──────╫──╫──╫──╫──╫──╫─┤M├───────────────────
      ├───┤├───┤      ║  ║  ║  ║  ║  ║ └╥┘┌─┐                
 q_7: ┤ H ├┤ H ├──────╫──╫──╫──╫──╫──╫──╫─┤M├─────

In [5]:
print("CNC Tableau Simulation Results:")

# Specify the directory where the QASM file is located
filename = f"dj_{n}_constant"
qasm_file = filename + ".qasm"
clifford_filename = filename + "_msi.qasm"

# Run the gadgetized circuit simulation.
counts, outputs, born_rule_estimates, shot_times = sim.run_qcm(qasm_dir, qasm_file, clifford_filename, shots=4096)

CNC Tableau Simulation Results:
------------------------------
Initial conditions for simulation:
------------------------------

Negativity:
 1

------------------------------
Outputs of Simulation:
------------------------------

Average time per shot: 0.000635 seconds

Distinct Outcome: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
Born Rule Estimate: 1.0



In [5]:
n = 3

print(f"Number of qubits: {n}\n")

# Test deutsch jozsa on constant:
qc = dj.run_deutsch_jozsa_test(n_qubits=n, oracle_type='balanced', constant_output=0)

print(qc,"\n")

# Simulate the circuit after decomposition
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=4096)
result = job.result()
counts = result.get_counts()

print("Qiskit Simulation Results:")
#print(counts)
percentages = [(k, v / sum(counts.values()))
                    for k, v in counts.items()]
print(f"Percentages: {percentages}\n")

Number of qubits: 3

Using a BALANCED oracle.
Quantum Circuit:

     ┌───┐     ┌───┐     ┌─┐                     
q_0: ┤ H ├──■──┤ H ├─────┤M├─────────────────────
     ├───┤  │  ├───┤     └╥┘┌─┐                  
q_1: ┤ H ├──■──┤ H ├──────╫─┤M├──────────────────
     ├───┤┌─┴─┐└───┘      ║ └╥┘          ┌───┐┌─┐
q_2: ┤ H ├┤ X ├───────────╫──╫────────■──┤ H ├┤M├
     ├───┤├───┤┌───┐┌───┐ ║  ║ ┌───┐┌─┴─┐└───┘└╥┘
q_3: ┤ H ├┤ S ├┤ S ├┤ H ├─╫──╫─┤ H ├┤ X ├──────╫─
     └───┘└───┘└───┘└───┘ ║  ║ └───┘└───┘      ║ 
c: 3/═════════════════════╩══╩═════════════════╩═
                          0  1                 2  

Conclusion: f(x) is BALANCED.
     ┌───┐     ┌───┐     ┌─┐                     
q_0: ┤ H ├──■──┤ H ├─────┤M├─────────────────────
     ├───┤  │  ├───┤     └╥┘┌─┐                  
q_1: ┤ H ├──■──┤ H ├──────╫─┤M├──────────────────
     ├───┤┌─┴─┐└───┘      ║ └╥┘          ┌───┐┌─┐
q_2: ┤ H ├┤ X ├───────────╫──╫────────■──┤ H ├┤M├
     ├───┤├───┤┌───┐┌───┐ ║  ║ ┌───┐┌─┴─┐└───┘└╥┘
q_3:

In [7]:
# Apply the 7T decomposition if applicable
qc_with_decomposition = toff.apply_toff_via_7t_decomposition(qc)

print("With decomposition:")
print(qc_with_decomposition)

qasm_code = qc_with_decomposition.qasm()

With decomposition:
      ┌───┐                                                                 »
 q_0: ┤ H ├────────────────────────────────■────────────────────────────────»
      ├───┤                                │            ┌───┐┌───┐┌───┐┌───┐»
 q_1: ┤ H ├───────■────────────────────────┼─────────■──┤ S ├┤ S ├┤ S ├┤ T ├»
      ├───┤┌───┐┌─┴─┐┌───┐┌───┐┌───┐┌───┐┌─┴─┐┌───┐┌─┴─┐├───┤├───┤├───┤├───┤»
 q_2: ┤ H ├┤ H ├┤ X ├┤ S ├┤ S ├┤ S ├┤ T ├┤ X ├┤ T ├┤ X ├┤ S ├┤ S ├┤ S ├┤ T ├»
      ├───┤└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
 q_3: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_4: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_5: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                 

In [8]:
# parent directory:
pardir = os.path.abspath(os.path.join(os.path.dirname('circuits'), '..'))
qasm_dir = os.path.join(pardir,'qasm_files')

filename = f'dj_{n}_balanced.qasm'
qasm_path = os.path.join(qasm_dir,filename)

# Replace 'tdg' with 's; s; s; t;'
modified_qasm_code = re.sub(r'\btdg\b', 's;\ns;\ns;\nt;', qasm_code)

with open(qasm_path, "w") as f:
    f.write(modified_qasm_code)

In [10]:
print("CNC Tableau Simulation Results:")

# Specify the directory where the QASM file is located
filename = f"dj_{n}_balanced"
qasm_file = filename + ".qasm"
clifford_filename = filename + "_msi.qasm"

# Run the gadgetized circuit simulation.
counts, outputs, born_rule_estimates, shot_times = sim.run_qcm(qasm_dir, qasm_file, clifford_filename, shots=2*4096)

CNC Tableau Simulation Results:
Saving keys for 7 qubits to file.

------------------------------
Initial conditions for simulation:
------------------------------

Negativity:
 4.8190599698530745

------------------------------
Outputs of Simulation:
------------------------------

Average time per shot: 0.001598 seconds

Distinct Outcome: (1, 0, 0, 0, 0, 0, 0, 0, 0, 1)
Born Rule Estimate: 0.04823766083104884

Distinct Outcome: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
Born Rule Estimate: 0.9512231410220241

Distinct Outcome: (1, 1, 0, 0, 0, 0, 0, 0, 0, 1)
Born Rule Estimate: -0.003529584938857232

Distinct Outcome: (0, 1, 0, 0, 0, 0, 0, 0, 0, 1)
Born Rule Estimate: 0.011177018973047903

